In [ ]:
from IPython.display import clear_output

In [ ]:
# Mount Google Drive
from google.colab import drive


drive.mount('/content/drive')

In [ ]:
# Install dependencies
!pip install TTS
!sudo apt-get install espeak-ng
!pip install onnx
!pip install onnxruntime

# STT
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install tabulate
!pip install pydub
!pip install transformers

# API-related dependencies
!pip install fastapi uvicorn pydantic pyngrok nest_asyncio
!pip install python-multipart

clear_output()

In [ ]:
# TTS-Related Imports
import IPython
import tempfile
import subprocess
import time
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits
from TTS.utils.audio.numpy_transforms import save_wav
import numpy as np


# STT-Related Imports
import io
import wave
import numpy as np
import whisper
import jiwer
import time
import pandas as pd
from tabulate import tabulate
from pydub import AudioSegment
import os
import joblib
import re
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F
from torch import nn, Tensor


# API-Related Imports
from fastapi import FastAPI,Response
from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware.gzip import GZipMiddleware
from fastapi.responses import JSONResponse
from fastapi.responses import StreamingResponse,FileResponse
from fastapi import FastAPI, UploadFile, File
import shutil
from pydantic import BaseModel
from IPython.display import Audio
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import base64
from typing import List, Optional


nest_asyncio.apply()

In [ ]:
from starlette.middleware.gzip import GZipMiddleware
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
app.add_middleware(GZipMiddleware, minimum_size=50)

## TTS

In [ ]:
# Load TTS model
live_config=VitsConfig()
live_config.load_json("/content/drive/MyDrive/NSMQ AI Project/Technical/TTS/Prof Elsie Kauffmann/VITS model/vits-elsie/traineroutput/vits_vctk-May-24-2023_11+05PM-23a7a9a3/config.json")
live_vits = Vits.init_from_config(live_config)
live_vits.load_onnx("/content/drive/MyDrive/NSMQ AI Project/Technical/TTS/Prof Elsie Kauffmann/VITS model/vits-elsie/elsie.onnx")

clear_output()

In [ ]:
def live_audio(text:str):
  text_inputs = np.asarray(
      live_vits.tokenizer.text_to_ids(text, language="en"),
      dtype=np.int64,
  )[None, :]
  audio = live_vits.inference_onnx(text_inputs,speaker_id=0)
  with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
    out_path = temp_file.name
  save_wav(wav=audio[0], path=out_path,sample_rate=22050)
  return out_path

In [ ]:
class LiveText(BaseModel):
  text: str

In [ ]:
@app.get('/synthesize-speech')
def onnx_audio(payload:LiveText):
  out_path=live_audio(payload.text)
  return FileResponse(out_path, media_type="audio/wav")

## STT

In [ ]:
# Load STT Model
# Load whisper model
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"

model = whisper.load_model("medium.en", device = DEVICE)

In [ ]:
def transcribe_audio(path_to_audio):
  """Loads whisper model to transcribe audio"""

  # Load audio
  audio = whisper.load_audio(path_to_audio)

  # Transcribe audio
  result = model.transcribe(audio)

  # Print transcript
  return result["text"]

In [ ]:
class AudioBytes(BaseModel):
  data: bytes
  filename: str

@app.get("/get-transcript")
async def get_transcript(audio: AudioBytes):
  try:
    decoded_data = base64.b64decode(audio.data)

    # Write bytes data to a .wav file
    with io.BytesIO(decoded_data) as audio_file:
        with wave.open(audio_file, "wb") as wav:
          wav.setnchannels(1)
          wav.setsampwidth(2)
          wav.setframerate(16000)

          # Write .wav files
          wav.writeframes(decoded_data)

    # Save the audio file with the custom name
    audio_filename = audio.filename
    with open(audio_filename, "wb") as file:
        file.write(decoded_data)

    transcript = transcribe_audio(audio_filename)
    os.remove(audio_filename)
    return {"transcript": transcript}
  except Exception as e:
    return {"error":str(e)}

# LM fine-tuning

### Mount Google drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install Dependencies

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

### Load Dataset

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/newdata1.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='/content/drive/MyDrive/newdata1.jsonl', split='train')

### Define dataset formatting function

In [ ]:
def formatting_func(example):
    text = f"### Question: {example['input']}\n ### Answer: {example['output']}"
    return text

### Authenticate with hugging face

In [ ]:
from huggingface_hub import login

login(token="")

### Load Base Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "TheBloke/Mistral-7B-Claude-Chat-GGUF"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

### Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

In [ ]:
max_length = 512 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt2)

In [ ]:
print(tokenized_train_dataset[1]['input_ids'])

### Test Base Model

In [ ]:
eval_prompt = """
Give me an NSMQ question
"""

In [ ]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input,max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True))

### Set Up LoRA

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

#### Start Training

In [ ]:
import transformers
from datetime import datetime

project = "journal-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=2,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training          # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

## LM

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "TheBloke/Mistral-7B-Claude-Chat-GGUF"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "TheBloke/Mistral-7B-Claude-Chat-GGUF/checkpoint-300")

In [ ]:
class Message(BaseModel):
    input: str
    output: Optional[str] = None

class InputModel(BaseModel):
    messages: List<Message]

class OutputModel(BaseModel):
    ai_response: str


In [ ]:
system_promt = """
You are a quiz mistress for the NSMQ quiz competition: 

This is how the competition occurs,
1. There are three teams in the competition
2. There are five rounds.
3 The first round consists of 24 questions with each school getting two sets of questions from each subject: Math, Chemistry, Physics and Biology.
Questions are asked for the different subjects one after the other with one question posed to each school from a set of questions from the same subject. 
For questions that don't require calculations, the time required is 10 seconds and for questions that require calculations the time is 30seconds.
4. The second round consists of 8 questions with each school at liberty of answering immediately they have an answer.
There are two questions each for each subject(Math, Chemistry, Physics and Biology) which any team can answer immediately the question is posed.
For every wrong answer given, there is a deduction of one point from the accummulated scores.  Th efirst team to answer a question gets the full 3 points.
The second team to make a correct attempt after the first team makes an attempt and gets it wrong, gets 2 points and if they make an unsuccessful attempt,
the last team after making a successful attempt gets one point.  
5. The third round also known as the problem of the day is such that a single question is posed to all three schools and each school has a maximum time of
4 minutes to present a solution to the problem. Marks are awarded based on the criteria and presentation of the solution apart from the final answer itself.
The total points to be accummulated is 10.
6. The fourth round round is also known as the True or False stage. Here, two questions each from each subject(Math, chemistry, Physicics and Biology)
are posed to each team. The questions are statements where each team is supposed to state whether the questions are True or False. 
For every answer given correctly, marks of two points are given. A failed attempt attracts a deduction of one point.
7. The fifth round also known as the riddle stage. There are four questions in this stage. There is one question for each subject(Math, chemistry, Physics
and Biology). The riddle is given as a set of clues which. The clues are given to the contestants one after the other. If the answer to the riddle 
is given on the first clue, 5 points are awarded to the contestant. If the answer is given on the second clue, 4 points are awarded. 3 points are awarded
on the subsequent clues. 



For questions that don't require calculations, the time required is 10 seconds and for questions that require calculations the time is 30seconds.
These are the procedures you will use to coordinate the quiz competition.
1. Your sole purpose is to train a single team.
3. The team you are training is team1.
2. Simulate the scores of other teams.
3. Ask the questions by following the processes outlined previously
4. Take the time into consideration when awarding marks. DO NOT give marks for late answers.
5. Do not give points for wrong answers.
6. If a team is not able to answer a question, pass the question to the next team. Stop passing the question if
the question has been answered correctly. Stop passing the question if the question has been answered wrongly by all teams.
7. Simulate the question and their responses for the other teams.
8. When a team answers a question correctly, award the points to the team and move to the next question. Direct the question to
the next team.
9. Give remarks to the previous teams response. The reponse should be "Correct for number of {number of points for answering that question}"
and for wrong answers, give the correct answer as a remark in the format "Wrong. The correct answer is {correct answer}"




Always return a json in this format:
"
{
"current_round":"current round number",
"current_question":"current question directed to a particular team",
"question:":"question",
"time":"time for the question in seconds",
"current_question":"current question directed to a particular team",
"quiz_mistress_remarks":"remarks",
"question_directed_at_team":"team",
"accumulated_points_for_team1":"points",
"accumulated_points_for_team2":"points",
"accumulated_points_for_team3":"points",
}
"
"""

In [ ]:



@app.post("/process", response_model=OutputModel)
async def process_data(input_data: InputModel):
    ai_response = input_data.messages[0]
    
    return OutputModel(ai_response=ai_response)

In [ ]:
!ngrok config add-authtoken # TO DO: Replace this comment with your ngronk token (can be obtained from your ngronk account).

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)